In [ ]:
import opentrons.execute
protocol = opentrons.execute.get_protocol_api('2.2')
protocol.home()

In [ ]:
TEMP_DECK = {
    'NAME': 'Temperature Module',
    'SLOT': 7
}
P200_MULTI = {
    'NAME': 'p300_multi',
    'POSITION': 'left'
}
FILTER_TIP_20 = [{
    'NAME': 'opentrons_96_filtertiprack_200ul',
    'SLOT': 1,
    'LABEL': 'Filter Tip S-1'
}]

ASPIRATE_DEPTH_BOTTOM = 2.00 #2mm from bottle
VOL_MASTER_MIX = 15 # Reaction volume
TEMP = 4
TOUCH_HEIGHT = -2.0
TOUCH_SPEED = 20.0

### Load all components

In [ ]:
temp_deck = protocol.load_module(TEMP_DECK['NAME'], location=TEMP_DECK['SLOT'])
reaction_plate = temp_deck.load_labware('biorad_96_wellplate_200ul_pcr',
                                        label='Reaction Plate')
reagent_plate = protocol.load_labware('biorad_96_wellplate_200ul_pcr',
                                      location=4,
                                      label='Reagent Plate')
reagent_mix = reagent_plate.columns()[0]
tip_20 = [protocol.load_labware(i['NAME'], location=i['SLOT'], label=i['LABEL'])
          for i in FILTER_TIP_20]
p200 = protocol.load_instrument(P200_MULTI['NAME'], P200_MULTI['POSITION'],
                                tip_racks=tip_20)
p200.well_bottom_clearance.aspirate = ASPIRATE_DEPTH_BOTTOM
p200.well_bottom_clearance.dispense = ASPIRATE_DEPTH_BOTTOM

In [ ]:
num_cols = len(reaction_plate.columns())

### Add master mix

In [ ]:
def add_master_mix(num_cols=1, pipette=None, source=None, dest=[]):
    # Transfer 15ul of reagent mix (reagent plate) to the pre-prepared reaction plate
    for c in range(num_cols):
        pipette.pick_up_tip()
        pipette.mix(repetitions=5, volume=VOL_MASTER_MIX, location=source[0])
        pipette.aspirate(volume=VOL_MASTER_MIX, location=source[0])
        pipette.dispense(volume=VOL_MASTER_MIX, location=dest[c][0])
        pipette.mix(repetitions=5, volume=VOL_MASTER_MIX)
        pipette.blow_out(location=dest[c][0])
        pipette.touch_tip(v_offset=TOUCH_HEIGHT, speed=TOUCH_SPEED)
        pipette.drop_tip()

temp_deck.set_temperature(celsius=TEMP)
add_master_mix(num_cols=num_cols, pipette=p200,
               source=reagent_mix, dest=reaction_plate.columns())